# Pulse Function Tests - Envelope System

This notebook tests the pulse envelope system that supports both cosine² and Gaussian envelopes:
- `envelope_type` parameter in `Pulse` class: 'cos2' or 'gaussian'
- `pulse_peak_time` = time where envelope is maximal
- `pulse_fwhm` = Full Width at Half Maximum parameter
- Pulse exists only in `[t_peak - fwhm, t_peak + fwhm]`
- Easy switching between envelope types via `SystemParameters.envelope_type`

Test scenarios:
- Envelope type comparison
- System-wide envelope control
- Fourier Transform spectral comparison

In [ ]:
# =============================
# IMPORT NECESSARY MODULES
# =============================
import numpy as np
import matplotlib.pyplot as plt

# Import from our project
from qspectro2d.visualization.mpl_tex_settings import *
from qspectro2d.core.pulse_sequences import PulseSequence, Pulse
from qspectro2d.core.pulse_functions import identify_non_zero_pulse_regions
from qspectro2d.core.pulse_functions import (
    pulse_envelope, Epsilon_pulse, E_pulse
)
from qspectro2d.core.system_parameters import SystemParameters
from qspectro2d.visualization.plotting import (
    Plot_pulse_envelope, Plot_fixed_tau_T
)

# Set matplotlib parameters for better plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

## Test 1: Envelope System - Easy Switching Between Types

Demonstrate how to easily switch between cosine² and Gaussian envelopes using the new system.

In [ ]:
# =============================
# ENVELOPE SYSTEM DEMONSTRATION
# =============================
print("ENVELOPE SYSTEM COMPARISON: COS² vs GAUSSIAN")

# Setup parameters
t_peak = 5.0
fwhm = 1.0

# Create both system types
systems = {
    'cos2': SystemParameters(pulse_fwhm=fwhm, envelope_type='cos2'),
    'gaussian': SystemParameters(pulse_fwhm=fwhm, envelope_type='gaussian')
}

# Create pulse sequences
pulse_params = (t_peak, np.pi)  # (time, phase)
pulse_seqs = {k: PulseSequence.from_pulse_specs(sys, [(0, pulse_params[0], pulse_params[1])]) for k, sys in systems.items()}

print(f"System envelope types:")
for name, seq in pulse_seqs.items():
    print(f"  {name.capitalize()}: {seq.pulses[0].envelope_type}")

# =============================
# TIME & FREQUENCY DOMAIN ANALYSIS
# =============================

# Time domain - for visualization
times = np.linspace(t_peak-2*fwhm, t_peak+2*fwhm, 1000)

# Frequency domain - for spectral analysis
dt = 0.1  # fs (time resolution)
t_spectral = np.arange(0, 300, dt)  # Extended range for frequency resolution
N = len(t_spectral)

# Calculate envelopes for both domains
envelopes = {}
spectra = {}

for name, seq in pulse_seqs.items():
    # Time domain
    envelopes[name] = {
        'envelope': [pulse_envelope(t, seq) for t in times],
        'E_field': [E_pulse(t, seq) for t in times],
        'full_field': [Epsilon_pulse(t, seq) for t in times]
    }
    
    # Frequency domain
    spectral_env = np.array([pulse_envelope(t, seq) for t in t_spectral])
    fft = np.fft.fft(spectral_env)
    freq = np.fft.fftfreq(N, dt)
    
    # Process spectrum data
    pos_idx = freq >= 0
    spectrum = np.abs(fft[pos_idx])
    spectrum /= np.max(spectrum)
    
    # Find spectral width
    max_val = np.max(spectrum)
    half_max = max_val / 2
    above_half = np.where(spectrum >= half_max)[0]
    fwhm = freq[pos_idx][above_half[-1]] - freq[pos_idx][above_half[0]] if len(above_half) > 0 else 0
    
    spectra[name] = {
        'freq': freq[pos_idx],
        'amplitude': spectrum,
        'fwhm': fwhm
    }

# =============================
# VISUALIZATION
# =============================

# Set up figure with 4 subplots in 2x2 layout (more compact)
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Colors and styles
colors = {'cos2': 'C0', 'gaussian': 'C1'}
linestyles = {'real': '-', 'imag': '--'}

# 1. Envelope comparison
ax = axes[0, 0]
for name, color in colors.items():
    ax.plot(times, envelopes[name]['envelope'], label=f'{name.capitalize()}', 
           color=color, linewidth=2)
ax.set_ylabel(r'Env$(t)$')
ax.set_title('Envelope Comparison')
ax.legend()

# 2. Electric field (RWA)
ax = axes[0, 1]
for name, color in colors.items():
    field = envelopes[name]['E_field']
    ax.plot(times, np.real(field), label=f'{name.capitalize()}-Re', color=color, linestyle='-')
    ax.plot(times, np.imag(field), label=f'-Im', color=color, linestyle='--')
ax.set_ylabel(r'$E(t)$ [arb. units]')
ax.set_title('Electric Field (RWA)')
ax.legend()

# 3. Full field with carrier
ax = axes[1, 0]
for name, color in colors.items():
    field = envelopes[name]['full_field']
    ax.plot(times, np.real(field), label=f'{name.capitalize()}-Re', color=color, linestyle='-')
    ax.plot(times, np.imag(field), label=f'-Im', color=color, linestyle='--')
ax.set_xlabel(r'Time $t$ [fs]')
ax.set_ylabel(r'$\varepsilon(t)$ [arb. units]')
ax.set_title('Electric Field (with carrier)')
ax.legend()

# 4. Frequency domain
ax = axes[1, 1]
for name, color in colors.items():
    ax.plot(spectra[name]['freq'], spectra[name]['amplitude'], 
           label=f'{name.capitalize()} (fwhm: {spectra[name]["fwhm"]:.3f})',
           color=color, linewidth=2)
ax.set_xlim(0, 3)  # Limit to relevant frequency range
ax.set_xlabel(r'Frequency $f$ [1/fs]')
ax.set_ylabel('Normalized amplitude')
ax.set_title('Envelope in Frequency Domain')
ax.legend()

plt.tight_layout()
plt.show()

# Print summary
print("\nSYSTEM USAGE:")
print("1. Global: SystemParameters(envelope_type='cos2' or 'gaussian')")
print("2. Individual: Pulse(..., envelope_type='...')")
print("3. Functions: pulse_envelope(), E_pulse(), Epsilon_pulse()")
print("✓ Benefit: Easy switching between envelope types")

# Test 2: Pulse regions

In [ ]:
# =============================
# TEST: identify_non_zero_pulse_regions
# =============================

# Define a test time array and a pulse sequence
test_times = np.linspace(-10, 10, 1000)
test_system = SystemParameters(pulse_fwhm=1.0, envelope_type='gaussian')
test_pulse_seq = PulseSequence.from_pulse_specs(test_system, [(0, -0.2, 0.0), (1, 2.2, 0.0), (2, 3.0, 0.0)])

# Get active regions where pulse envelope is non-zero
active_regions = identify_non_zero_pulse_regions(test_times, test_pulse_seq)
active_times = test_times[active_regions]  # Only times where pulses are active

# Calculate the envelope across all times (now works with array input)
envelope_values = pulse_envelope(test_times, test_pulse_seq)

# Plot the envelope and highlight active regions
plt.figure(figsize=(10, 4))
plt.plot(test_times, envelope_values, 'b-', label='Pulse Envelope')
plt.fill_between(test_times, envelope_values, where=active_regions, 
                 color='yellow', alpha=0.5, label='Non-zero region')
plt.xlabel('Time [fs]')
plt.ylabel('Envelope Amplitude')
plt.title('Pulse Envelope with Highlighted Non-zero Regions')
plt.grid(False)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# =============================
# TEST NEW E0 CALCULATION FEATURE
# =============================

print("Testing the new E0 calculation feature...")
print("=" * 50)

### Test with the existing pulse sequence
test_times_for_E0 = np.linspace(-2, 10, 200)  # Extended time range

print(f"Pulse sequence contains {len(test_pulse_seq.pulses)} pulses:")
for i, pulse in enumerate(test_pulse_seq.pulses):
    start, end = pulse.active_time_range
    print(f"  Pulse {i}: t_peak={pulse.pulse_peak_time:.1f}, fwhm={pulse.pulse_fwhm:.1f}")
    print(f"           Active range: [{start:.1f}, {end:.1f}], Amplitude: {pulse.pulse_amplitude:.3f}")

print("\nTesting E0 calculation at specific times:")
print("-" * 40)

### Test at different times
test_specific_times = [0.0, 2.0, 2.5]

for t in test_specific_times:
    # Get comprehensive field info
    field_info = test_pulse_seq.get_field_info_at_time(t)
    
    print(f"Time t = {t:.1f}:")
    print(f"  Active pulses: {field_info['num_active_pulses']}")
    print(f"  Total E0: {field_info['total_amplitude']:.4f}")
    
    if field_info['num_active_pulses'] > 0:
        print(f"  Active pulse indices: {field_info['pulse_indices']}")
        print(f"  Individual amplitudes: {[f'{amp:.4f}' for amp in field_info['individual_amplitudes']]}")
    print()

### Plot E0 vs time
print("Plotting E0(t) - Total electric field amplitude over time...")

E0_values = np.array([test_pulse_seq.get_total_amplitude_at_time(t) for t in test_times_for_E0])

plt.figure(figsize=(12, 6))

### Plot E0 vs time
plt.subplot(2, 1, 1)
plt.plot(test_times_for_E0, E0_values, 'C0-', linewidth=2, label=r'$E_0(t)$ - Total Amplitude')
plt.xlabel(r'Time $t$ [fs]')
plt.ylabel(r'$E_0(t)$ [a.u.]')
plt.title('Total Electric Field Amplitude vs Time')
plt.grid(True, alpha=0.3)
plt.legend()

### Mark individual pulse peak times
for i, pulse in enumerate(test_pulse_seq.pulses):
    plt.axvline(pulse.pulse_peak_time, color=f'C{i+1}', linestyle='--', alpha=0.7, 
                label=f'Pulse {i} peak (t={pulse.pulse_peak_time:.1f})')

plt.legend()

### Plot number of active pulses vs time
plt.subplot(2, 1, 2)
num_active = np.array([test_pulse_seq.get_field_info_at_time(t)['num_active_pulses'] for t in test_times_for_E0])
plt.step(test_times_for_E0, num_active, 'C3-', linewidth=2, where='mid', label='Number of Active Pulses')
plt.xlabel(r'Time $t$ [fs]')
plt.ylabel('Number of Active Pulses')
plt.title('Number of Active Pulses vs Time')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

print("E0 calculation test completed successfully!")